In [0]:
!pip install lightfm

In [0]:
import pandas as pd # pandas for data manipulation
import numpy as np # numpy for sure
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix # for constructing sparse matrix
from scipy.sparse import save_npz
# lightfm 
from lightfm import LightFM # model
from lightfm.evaluation import auc_score,precision_at_k,recall_at_k
import pickle

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%%time
path = '/content/drive/My Drive/Insight/'
orders = pd.read_csv(path+'orders.csv')
order_products__prior = pd.read_csv(path+'order_products__prior.csv')
order_products__train = pd.read_csv(path+'order_products__train.csv')
products = pd.read_csv(path+'products.csv')

CPU times: user 8.86 s, sys: 2.36 s, total: 11.2 s
Wall time: 11.9 s


In [0]:
# visualize headers of all tables
table_names = ['orders','order_products__prior','order_products__train','products']
for i,df in enumerate([orders,order_products__prior,order_products__train,products]):
    print(table_names[i])
    print(len(df))
    display(df.head())

orders
3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


order_products__prior
32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


order_products__train
1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


products
49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
len(order_products__train['order_id'].unique())

131209

In [0]:
# Normalize user_product matrix by number of items from each user
normalize_freq = True

In [0]:
%%time

# For each user, count number of products they've purchased in prio

# Get prior orders by user
users_order_prior = orders.loc[orders['eval_set'].map(lambda x: x in ['prior'])]

# Merge products in prior orders to the user for each order
print(len(order_products__prior))
display(order_products__prior.head(5))
user_order_products_prior_train = order_products__prior.merge(users_order_prior[['order_id','user_id']],
                                                        on='order_id')
print(len(user_order_products_prior_train))
display(user_order_products_prior_train.head(5))

# Check join worked
display(users_order_prior.loc[users_order_prior['order_id']==2]) # does this return user 202279?

# Count frequency of each product bought by each user. These will be the "ratings" for lightfm
user_products_freq_train_long = user_order_products_prior_train[['user_id','product_id']]\
                                .groupby(['user_id','product_id']).size()
# display(user_products_freq_train_long.head(25))

# Reset index to remove MultiIndex
user_products_freq_train_long = user_products_freq_train_long.reset_index()
user_products_freq_train_long.columns = ['user_id','product_id','freq']
display(user_products_freq_train_long.head(25))

32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


32434489


,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,9327,3,0,202279
3,2,45918,4,1,202279
4,2,30035,5,0,202279


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3355525,2,202279,prior,3,5,9,8.0


,user_id,product_id,freq
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3
5,1,13176,2
6,1,14084,1
7,1,17122,1
8,1,25133,8
9,1,26088,2


CPU times: user 19.1 s, sys: 1.41 s, total: 20.5 s
Wall time: 20.6 s


In [0]:
%%time
# Repeat above cell to make a test dataset made up of including a user's most recent order 
#  (in the Instacart data, known as "train" dataset of 131209 users)

# Get prior orders by user
users_order_most_recent = orders.loc[orders['eval_set'].map(lambda x: x in ['train'])]

# Merge products in prior orders to the user for each order
print(len(order_products__train))
display(order_products__train.head(5))
user_order_products_recent_test = order_products__train.merge(users_order_most_recent[['order_id','user_id']],
                                                        on='order_id')
print(len(user_order_products_recent_test))
display(user_order_products_recent_test.head(5))

# Check join worked
display(users_order_most_recent.loc[users_order_most_recent['order_id']==1]) # does this return user 112108?

# Count frequency of each product bought by each user. These will be the "ratings" for lightfm
user_products_freq_test_long = user_order_products_recent_test[['user_id','product_id']]\
                                .groupby(['user_id','product_id']).size()
# display(user_products_freq_train_long.head(25))

# Reset index to remove MultiIndex
user_products_freq_test_long = user_products_freq_test_long.reset_index()
user_products_freq_test_long.columns = ['user_id','product_id','freq']
# display(user_products_freq_test_long.head(25))

# make test dataframe include not just the most recent order
    
user_products_freq_test_long = user_products_freq_test_long\
    .merge(user_products_freq_train_long.rename(
            columns = {"freq" : "previous_freq"}), 
           how = "left").fillna(0)
user_products_freq_test_long["freq"] = \
  user_products_freq_test_long["previous_freq"] + \
  user_products_freq_test_long["freq"]
user_products_freq_test_long.drop(columns = ["previous_freq"], inplace = True)

display(user_products_freq_test_long.head(25))


1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


1384617


,order_id,product_id,add_to_cart_order,reordered,user_id
0,1,49302,1,1,112108
1,1,11109,2,1,112108
2,1,10246,3,0,112108
3,1,49683,4,0,112108
4,1,43633,5,1,112108


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1868044,1,112108,train,4,4,10,9.0


,user_id,product_id,freq
0,1,196,11.0
1,1,10258,10.0
2,1,13032,4.0
3,1,25133,9.0
4,1,26088,3.0
5,1,26405,3.0
6,1,27845,1.0
7,1,38928,2.0
8,1,39657,2.0
9,1,46149,4.0


CPU times: user 6.45 s, sys: 42.4 ms, total: 6.5 s
Wall time: 6.51 s


In [0]:
# Turn long user/product frequency table into a wide matrix

# This fails so try to do it using coo_matrix() below
# user_products_freq_train_wide = user_products_freq_train_long.unstack(fill_value=0)

# Row index will be user_id -1 (because no row_id=0)
row_ind = user_products_freq_train_long['user_id'].values-1
num_rows = max(user_products_freq_train_long['user_id'].unique())

# Column index will be prod_id -1 (because no prod_id=0)
col_ind = user_products_freq_train_long['product_id'].values-1
num_cols = max(user_products_freq_train_long['product_id'].unique())

# Frequency data are cell values
freq = user_products_freq_train_long['freq'].values

# Create sparse matrix
user_prod_interaction_train = coo_matrix((freq, (row_ind, col_ind)), shape = (num_rows, num_cols))

user_prod_interaction_train


<206209x49688 sparse matrix of type '<class 'numpy.int64'>'
	with 13307953 stored elements in COOrdinate format>

In [0]:
# Turn long user/product frequency table into a wide matrix for test data

# Row index will be user_id -1 (because no row_id=0)
row_ind = user_products_freq_test_long['user_id'].values-1
num_rows = max(user_products_freq_test_long['user_id'].unique())

# Column index will be prod_id -1 (because no prod_id=0)
col_ind = user_products_freq_test_long['product_id'].values-1
num_cols = max(user_products_freq_test_long['product_id'].unique())

# Frequency data are cell values
freq = user_products_freq_test_long['freq'].values

# Create sparse matrix
user_prod_interaction_test = coo_matrix((freq, (row_ind, col_ind)), shape = (num_rows, num_cols))

user_prod_interaction_test

<206209x49688 sparse matrix of type '<class 'numpy.float64'>'
	with 1384617 stored elements in COOrdinate format>

In [0]:
# Calculate sparsity of matrices

#train matrix
print('{}% of cells are non-zero in train'.format(100*13307953/(206209*49688)))

#test matrix
print('{}% of cells are non-zero in test'.format(100*1384617/(206209*49688)))

0.12988294458744037% of cells are non-zero in train
0.01351358342532679% of cells are non-zero in test


# Model fitting

In [0]:
%%time
# Initialising model with warp loss function
model = LightFM(loss = "warp") #rmse ?

# this fails on my local machine, but runs here!
model.fit(user_prod_interaction_train,epochs=1) #normalize dataset. #different approaches to the matrix factoralization

CPU times: user 32.4 s, sys: 61 ms, total: 32.5 s
Wall time: 32.5 s


In [0]:
%%time

# Validation of model using precision at k
# precisionatk_train = precision_at_k(model = model, 
#                         test_interactions = user_prod_interaction_train,
#                         check_intersections=False)
precisionatk_test = precision_at_k(model = model, 
                        test_interactions = user_prod_interaction_test,
                        train_interactions = user_prod_interaction_train, 
                        check_intersections=False)
# display(precisionatk_train.mean()) #0.26
display(precisionatk_test.mean()) #0.1

0.26239207

CPU times: user 20min 3s, sys: 240 ms, total: 20min 3s
Wall time: 20min 4s


In [0]:
%%time

# Calculate precision at k for multiple ks
for i in [1,2,3,4,5,10,15,20]:
  precisionatk_test = precision_at_k(model = model, 
                          test_interactions = user_prod_interaction_test,
                          train_interactions = user_prod_interaction_train, 
                          k=i,
                          check_intersections=False)
  print('precision at {} = {}'.format(i,precisionatk_test.mean()))

precision at 1 = 0.21357528865337372
precision at 2 = 0.18258275091648102
precision at 3 = 0.16255490481853485
precision at 4 = 0.14644193649291992
precision at 5 = 0.1330914944410324
precision at 10 = 0.09420084953308105
precision at 15 = 0.07652930915355682
precision at 20 = 0.0663849264383316
CPU times: user 47min 30s, sys: 494 ms, total: 47min 30s
Wall time: 47min 32s


In [0]:
%%time
# Validation of model using recall at k
# recallatk_train = recall_at_k(model = model, 
#                         test_interactions = user_prod_interaction_train,
#                         check_intersections=False)
recallatk_test = recall_at_k(model = model, 
                        test_interactions = user_prod_interaction_test,
                        train_interactions = user_prod_interaction_train, 
                        check_intersections=False)
# display(recallatk_train.mean())
display(recallatk_test.mean()) 

0.09037403589028865

CPU times: user 6min, sys: 82.6 ms, total: 6min
Wall time: 6min


In [0]:
%%time

# Validation of model using auc
# auc_score_train = auc_score(model = model, 
#                         test_interactions = user_prod_interaction_train,
#                         check_intersections=False)
auc_score_test = auc_score(model = model, 
                        test_interactions = user_prod_interaction_test,
                        train_interactions = user_prod_interaction_train, 
                        check_intersections=False)
# display(auc_score_train.mean())
display(auc_score_test.mean())

0.9485358

CPU times: user 6min, sys: 74.1 ms, total: 6min
Wall time: 6min


In [0]:
#other validation metrics
#ranking metrics NDCG (normalized discounted cumulative gain)

In [0]:
# Save model file and user_product_interaction_matrix for recommendations
with open('model_lightfm_v2.pkl', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)
save_npz('user_prod_interaction',user_prod_interaction_train)

In [0]:
user_id_single = [100]
row_ixs = [user_id-1 for user_id in user_id_single] #subtract one to get indices

# Look for this user in the user_prod_interaction matrix
col_ixs = user_prod_interaction_train.tocsr()[row_ixs].indices 
print(col_ixs) #these are the column indices in the matrix. 

# Need to add 1 to get product ids.
user_prod_ids = col_ixs+1

# This user purchased these items:
purch_products = products.loc[products['product_id'].isin(user_prod_ids)]
print('Purchased products:')
display(purch_products)

# Scores from LightFM model prediction for all products:
scores = model.predict(user_ids = row_ixs, item_ids = np.arange(user_prod_interaction_train.shape[1]))

print(scores)
# argsort returns column indices from the matrix, but we're using iloc so no need to increment
recom_products = products.iloc[np.argsort(-scores)] 

print('Recommended products:')
display(recom_products.head(5))



NameError: ignored

In [0]:
#How is it getting the recommendation?
# best recommendation may be dot product of user with all items. cosine similarity.

In [0]:
# vectors for embeddings.
